# Facebook爬蟲
這裡簡單介紹一下爬蟲的邏輯再開始爬進入正題吧！
- 爬取文章內容
 - 貼文人ID
 - 貼文人暱稱
 - 貼文時間
 - 貼文類型
 - 貼文內容
 - 心情：分為Like, Haha, Sad, Wow, Angry...etc.
 - 留言數
 - 分享數
 - 貼文連結


- 爬取留言內容
 - 留言人ID
 - 留言人暱稱
 - 留言時間
 - 回覆對象ID
 - 回覆對象暱稱
 - 貼文連結
 

# 載入使用套件

In [7]:
import pandas as pd
import re, time, requests, datetime, gc
from selenium import webdriver
from bs4 import BeautifulSoup

# 經常用到的函數
Facebook請用戶「登入/註冊」的彈窗會反覆出現，在這裡先寫好嘗試點擊的函數提供使用

In [8]:
# 點擊不要現在註冊帳號
def ClickNotNow():
    try:
        driver.find_element_by_xpath('//a[@id="expanding_cta_close_button"]').click()
    except:
        time.sleep(0.1)

# 截取塗鴉墻貼文資訊並比對先前資料
如果每次都把所有貼文抓下來，無疑會浪費程式效能，在這裡我將載入先前的資料，並將各篇貼文分成以下三類：

- **新貼文**：截取貼文內容與留言
- **舊貼文但有新留言**：截取所有留言但只將新留言加入資料庫
- **舊貼文也無新留言**：不處理

可想而知，如果設定排程每天執行，大多數資料都會是「舊貼文也無新留言」的類型，加入這個判斷機制將大幅節省系統效能

In [9]:
# 截取塗鴉墻上貼文的留言數
def Wall_PostComments(i):
    try:
        Comments = i.find('a', {'data-testid':'UFI2CommentsCount/root'}).text.split(' ',2)[0]
        if 'K' in Comments:
            Comments = int(float(Comments.split('K')[0])*1000)
        else:
            Comments = int(Comments)
    except:
        Comments = 0
    return Comments

# 截取塗鴉墻上貼文的連結
def Wall_PostLink(i):
    Link = 'https://www.facebook.com' + i.find('a',{'class':'_5pcq'}).attrs['href'].split('?',2)[0]
    return Link

# 截取塗鴉墻上貼文的發佈時間
def Wall_PostTime(i):
    try:
        Time = i.find('abbr').attrs['title']
        Time = datetime.datetime.strptime(Time, '%m/%d/%y, %I:%M %p')
        Time = Time.strftime("%Y-%m-%d %H:%M")
    except:
        Time = 'Not Post'
    return Time

def CarwlList(urls, n, Posts):
    CheckList = pd.DataFrame(columns = ['Link','Time','Comments']) 
    for i in urls:
        driver.get(i)        
        for i in range(n):
            try:
                time.sleep(1)
                driver.find_element_by_css_selector('a.pam.uiBoxLightblue.uiMorePagerPrimary').click() # 加載更多貼文的按鈕
            except:
                time.sleep(1)
            # 這裡會跳出要我們登入的大畫面，找到「稍後再說」的按鈕並點擊
            ClickNotNow()
            soup = BeautifulSoup(driver.page_source)
            for i in soup.find_all('div', {'class':'_5pcr userContentWrapper'}):
                CheckList = pd.concat([CheckList, pd.DataFrame(data = [{'Link':Wall_PostLink(i),
                                                                        'Time':Wall_PostTime(i),
                                                                        'Comments':Wall_PostComments(i)}],
                                                               columns = ['Link','Time','Comments'])],
                                       ignore_index = True)
    
    CarwlList = pd.merge(left = CheckList,
                         right = Posts.loc[:,['Link','Comments']],
                         how='left',
                         on='Link',
                         suffixes=('_c', '_p'),
                         indicator=True)
    CarwlList = pd.concat([CarwlList.loc[CarwlList._merge == 'left_only', :], # 新貼文
                           CarwlList.loc[CarwlList.Comments_c > CarwlList.Comments_p, :]], # 舊貼文但有新留言
                          ignore_index=True)
    CarwlList = CarwlList.drop_duplicates(subset = 'Link',
                                          keep= 'first',
                                          inplace = False)
    return CarwlList

# 爬取特定貼文的內容與留言
## 展開貼文與留言

In [10]:
# 關閉彈窗
def ClosePopup():
    time.sleep(1)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(1)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    ClickNotNow()
    driver.find_element_by_xpath('//div[@class="_5pcr userContentWrapper"]//a[@data-testid="UFI2CommentsCount/root"]').click()
    time.sleep(1)
    ClickNotNow()
    time.sleep(0.5)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(0.5)
    ClickNotNow()
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(0.5)
    ClickNotNow()

# 選擇按照New或Oldest來顯示留言(選擇最相關或所有留言都不會會真的顯示所有留言)
def ClickOldest():
    time.sleep(1)
    driver.find_element_by_xpath('//a[@data-testid="UFI2ViewOptionsSelector/link"]').click()
    time.sleep(1)
    try:
        driver.find_element_by_partial_link_text('Comments shown in chronological order with the oldest comments at the top.').click()
    except:
        try:
            driver.find_element_by_partial_link_text('New comments and those with new replies appear at the top.').click()
        except:
            print('Plz, Check this post arragne type!')
    time.sleep(1)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    ClickNotNow()
    time.sleep(1)

# 偵測是否有「更多留言」（第一層），若有則點擊
def MoreComments():
    k = len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]'))
    # 卡住的次數超過100次就停止
    l = 0
    while (k != 0) & (l <= 300):
        print('Click More Comments： ' + str(l) + ' times.')
        try:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            # 若彈出註冊視窗點擊not now!
            ClickNotNow()
            driver.find_element_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]').click()
            time.sleep(1)
            k = len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]'))
        except:
            time.sleep(0.1)
        finally:
            l += 1
    time.sleep(1)

# 偵測是否有「更多回覆」（第二層），若有則點擊
def MoreReplies():
    k = len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_1"]'))
    # 卡住的次數超過100次就停止
    l = 0
    while (k != 0) & (l <= 300):
        l += 1
        print('Click More Replies： ' + str(l) + ' times.')
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        driver.find_element_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_1"]').click()
        time.sleep(0.5)
        k = len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_1"]'))
    time.sleep(1)
    
# 偵測是否有「展開留言」（第二層留言內容），若有則點擊
def SeeMore():
    k = len(driver.find_elements_by_xpath('//a[@class="_5v47 fss"]'))
    # 卡住的次數超過100次就停止
    l = 0
    while (k != 0) & (l <= 1000):
        l += 1
        print('Click See More ： ' + str(l) + ' times.')
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        driver.find_element_by_xpath('//a[@class="_5v47 fss"]').click()
        time.sleep(0.5)
        k = len(driver.find_elements_by_xpath('//a[@class="_5v47 fss"]'))
    time.sleep(1)
    
# 組合以上函數，將貼文內容展開
def PostExpand():
    ClosePopup()
    # ClickOldest() # 展開留言會有無法加載更多留言的問題，暫時先不使用
    MoreComments()
    MoreReplies()
    SeeMore()

## 爬取貼文內容與留言資訊

In [11]:
# 貼文留言數
def Post_Name(userContent):
    return userContent.find('img').attrs['aria-label']

# 貼文留言數
def Post_ID(userContent):
    return userContent.find('a', {'class':'_5pb8 o_c3pynyi2g _8o _8s lfloat _ohe'}).attrs['href'].split('/?ref')[0].split('https://www.facebook.com/')[1]

# 貼文時間
def Post_Time(userContent):
    try:
        Time = userContent.find('abbr').attrs['title']
        Time = datetime.datetime.strptime(Time, '%m/%d/%y, %I:%M %p')
        Time = Time.strftime("%Y-%m-%d %H:%M")
    except:
        Time = "Error, Please check this post's condiction!" 
    return Time

# 貼文內容
def Post_Content(userContent):
    try:
        Content = userContent.find('div', {'class':'_5pbx userContent _3576'}).text
    except:
        Content = "There's No Text Content!"
    return Content

# 貼文留言數
def Post_Comments(userContent):
    try:
        CommentCounts = userContent.find('a', {'data-testid':'UFI2CommentsCount/root'}).text.split(' ',2)[0]
        if 'K' in CommentCounts:
            CommentCounts = int(float(CommentCounts.split('K')[0])*1000)
        else:
            CommentCounts = int(CommentCounts)
    except:
        CommentCounts = 0 
    return CommentCounts

# 貼文分享數
def Post_Shares(userContent):
    try:
        ShareCounts = userContent.find('span', {'class':'_355t _4vn2'}).text.split(' ',2)[0]
        if 'K' in ShareCounts:
            ShareCounts = int(float(ShareCounts.split('K')[0])*1000)
        else:
            ShareCounts = int(ShareCounts)
    except:
        ShareCounts = 0
    return ShareCounts

# 貼文按讚心情數
def Post_Likes(userContent):
    try:
        LikeCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_LIKE'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in LikeCounts:
            LikeCounts = int(float(LikeCounts.split('K')[0])*1000)
        else:
            LikeCounts = int(LikeCounts)
    except:
        LikeCounts = 0
    return LikeCounts

# 貼文愛心心情數
def Post_Loves(userContent):
    try:
        LoveCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_LOVE'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in LoveCounts:
            LoveCounts = int(float(LoveCounts.split('K')[0])*1000)
        else:
            LoveCounts = int(LoveCounts)
    except:
        LoveCounts = 0
    return LoveCounts

# 貼文哈哈心情數
def Post_Hahas(userContent):
    try:
        HahaCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_HAHA'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in HahaCounts:
            HahaCounts = int(float(HahaCounts.split('K')[0])*1000)
        else:
            HahaCounts = int(HahaCounts)
    except:
        HahaCounts = 0
    return HahaCounts

# 貼文Wow心情數
def Post_Wows(userContent):
    try:
        WowCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_WOW'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in WowCounts:
            WowCounts = int(float(WowCounts.split('K')[0])*1000)
        else:
            WowCounts = int(WowCounts)
    except:
        WowCounts = 0
    return WowCounts

# 貼文Sad心情數
def Post_Sads(userContent):
    try:
        SadCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_SORRY'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in SadCounts:
            SadCounts = int(float(SadCounts.split('K')[0])*1000)
        else:
            SadCounts = int(SadCounts)
    except:
        SadCounts = 0
    return SadCounts

# 貼文Angry心情數
def Post_Angrys(userContent):
    try:
        AngryCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_ANGER'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in AngryCounts:
            AngryCounts = int(float(AngryCounts.split('K')[0])*1000)
        else:
            AngryCounts = int(AngryCounts)
    except:
        AngryCounts = 0
    return AngryCounts

# 文章內容與互動摘要
def PostInfo(soup):
    # 貼文區
    userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
    PostContent = pd.DataFrame(data = [{'Name':Post_Name(userContent),
                                        'ID':Post_ID(userContent),
                                        'Time':Post_Time(userContent),
                                        'Content':Post_Content(userContent),
                                        'Comments':Post_Comments(userContent),
                                        'Shares':Post_Shares(userContent),
                                        'Likes':Post_Likes(userContent),
                                        'Loves':Post_Loves(userContent),
                                        'Hahas':Post_Hahas(userContent),
                                        'Wows':Post_Wows(userContent),
                                        'Sads':Post_Sads(userContent),
                                        'Angrys':Post_Angrys(userContent),
                                        'Updatetime':datetime.datetime.now().strftime("%Y-%m-%d %H:%M"),
                                        'Link':driver.current_url}],
                            columns = ['Name', 'ID', 'Time', 'Content', 'Comments', 'Shares', 'Likes', 'Loves', 'Hahas', 'Wows', 'Sads', 'Angrys', 'Updatetime', 'Link'])
    return PostContent

# 留言內容
def Comment_Content(element):
    try:
        Content = element.find('span', {'dir':'ltr'}).text
    except:
        Content = 'img'
    return Content

def CommentsInfo(soup):  
    PostComments = pd.DataFrame()
    userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
    try:
        for i in userContent.select('ul._7a9a > li'):
            # 先抓留言並放在Comment
            Comment = pd.DataFrame(data=[{'ID':i.find('a', {'class':' _3mf5 _3mg0'}).attrs['data-hovercard'].split('id=',2)[1],
                                          'Name':i.find('img').attrs['alt'],
                                          'Time':datetime.datetime.strptime(i.find('abbr',{'class':'livetimestamp'}).attrs['data-tooltip-content'], '%A, %B %d, %Y at %I:%M %p').strftime("%Y-%m-%d %H:%M"),
                                          'Content':Comment_Content(i),
                                          'RepID':userContent.find('div', {'class':'_5pcp _5lel _2jyu _232_'}).attrs['id'].split(';')[0].split('feed_subtitle_')[-1],
                                          'RepName':userContent.find('img').attrs['aria-label'],
                                          'Link':driver.current_url}],
                                   columns = ['ID', 'Name', 'Time', 'Content', 'RepID', 'RepName', 'Link'])
            PostComments = pd.concat([PostComments, Comment], ignore_index=True)
            # 留言的留言
            for j in i.findAll('div', {'data-testid':'UFI2Comment/root_depth_1'}):
                Comment = pd.DataFrame(data=[{'ID':j.find('a', {'class':' _3mf5 _3mg1'}).attrs['data-hovercard'].split('id=',2)[1],
                                              'Name':j.find('img').attrs['alt'],
                                              'Time':datetime.datetime.strptime(j.find('abbr',{'class':'livetimestamp'}).attrs['data-tooltip-content'], '%A, %B %d, %Y at %I:%M %p').strftime("%Y-%m-%d %H:%M"),
                                              'Content':Comment_Content(j),
                                              'RepID':i.find('a', {'class':' _3mf5 _3mg0'}).attrs['data-hovercard'].split('id=',2)[1],
                                              'RepName':i.find('img').attrs['alt'],
                                              'Link':driver.current_url}],
                                       columns = ['ID', 'Name', 'Time', 'Content', 'RepID', 'RepName', 'Link'])
                PostComments = pd.concat([PostComments, Comment], ignore_index=True)
        PostComments['Updatetime'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
    except:
        print('Crawl Comments Failed!')
    return PostComments

# 更新貼文與留言資訊
將最新抓到的貼文併回Post表格，並以ID與Time作為關鍵(key)值，取最新抓取的資料(Updatetime)

In [12]:
def UpdateData(DateFrame_o,DateFrame_n):
    DataFrame = pd.concat([DateFrame_o, DateFrame_n], ignore_index=True)
    DataFrame = DataFrame.sort_values(by = 'Updatetime', ascending = False)
    DataFrame = DataFrame.drop_duplicates(subset = ['ID', 'Time', 'Content'],
                                          keep= 'first',
                                          inplace = False)
    return DataFrame

# 爬資料
## 讀取先前資料
若在桌面偵測不到檔案，就創造新的dataframe

In [13]:
try:
    Posts = pd.read_pickle('C:/Users/TL_Yu/Desktop/Posts.plk')
except:
    Posts = pd.DataFrame(columns=['Name', 'ID', 'Time', 'Content', 'Comments', 'Shares', 'Likes', 'Loves', 'Hahas', 'Wows', 'Sads', 'Angrys', 'Updatetime', 'Link'])
try:
    Comments = pd.read_pickle('C:/Users/TL_Yu/Desktop/Comments.plk')
except:
    Comments = pd.DataFrame(columns = ['ID', 'Name', 'Time', 'Content', 'RepID', 'RepName', 'Link', 'Updatetime'])

print(Posts.shape)
print(Comments.shape)
urls = ['https://www.facebook.com/tstartel/',
        'https://www.facebook.com/chtmobile/',
        'https://www.facebook.com/taiwanmobile/',
        'https://www.facebook.com/fareastone/',
        'https://www.facebook.com/Aptg.tw/']

(210, 14)
(40718, 8)


## 產出待爬清單

In [5]:
driver = webdriver.Chrome()
driver.get('https://www.facebook.com/')
time.sleep(1)
driver.find_element_by_partial_link_text('English').click()

In [14]:
CarwlList = CarwlList(urls=urls, n=30, Posts = Posts)
CarwlList

,Link,Time,Comments_c,Comments_p,_merge
0,https://www.facebook.com/tstartel/photos/a.413...,2019-04-21 21:00,26,NaN,left_only
1,https://www.facebook.com/tstartel/photos/a.239...,2019-04-19 20:51,67,NaN,left_only
6,https://www.facebook.com/tstartel/posts/318084...,2019-04-07 21:11,109,NaN,left_only
13,https://www.facebook.com/tstartel/videos/58966...,2019-03-31 21:03,0,NaN,left_only
18,https://www.facebook.com/tstartel/posts/315158...,2019-03-26 17:19,25,NaN,left_only
19,https://www.facebook.com/tstartel/videos/40234...,2019-03-24 20:30,0,NaN,left_only
44,https://www.facebook.com/tstartel/videos/42242...,2019-02-21 04:00,0,NaN,left_only
45,https://www.facebook.com/tstartel/posts/307457...,2019-02-20 15:02,38,NaN,left_only
46,https://www.facebook.com/tstartel/posts/307248...,2019-02-19 20:00,38,NaN,left_only
65,https://www.facebook.com/tstartel/videos/77488...,2019-02-14 22:09,2000,NaN,left_only


如同上面的說明，當CheckLists「merge」欄位為「left_only」時表示該貼文是新貼文；

而當「merge」欄位為「both」，但「Comments_c」的值大於「Comments_p」時，表示該貼文是舊資料但有新留言。

以下將截取這兩類的貼文作為爬取清單

## 執行爬蟲

In [15]:
for i in CarwlList.Link:
    print('Dealing with: ' + i)
    driver.get(i)
    try:
        PostExpand()
        print('Expand Succed!')
        try:
            soup = BeautifulSoup(driver.page_source)
            time.sleep(1)
            nPost = PostInfo(soup)
            nComments = CommentsInfo(soup)
            Posts = UpdateData(DateFrame_o = Posts, DateFrame_n = nPost)
            print('Update PostInfo complete!')
            Comments = UpdateData(DateFrame_o = Comments,DateFrame_n = nComments)
            print('Update CommentsInfo complete!')
        except:
            print('Update Post or CommentsInfo Failed!')
    except:
        print('This post can not expand')
    gc.collect()
    print('Time Log: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + '\n\n------------------')

Dealing with: https://www.facebook.com/tstartel/photos/a.413576055335114/3214817948544230/
Click More Comments： 0 times.
Click More Replies： 1 times.
Click More Replies： 2 times.
This post can not expand
Time Log: 2019-05-29 14:22:23

------------------
Dealing with: https://www.facebook.com/tstartel/photos/a.2393598470666186/3209498659076159/
Click More Comments： 0 times.
Click More Replies： 1 times.
Click More Replies： 2 times.
This post can not expand
Time Log: 2019-05-29 14:22:32

------------------
Dealing with: https://www.facebook.com/tstartel/posts/3180842608608431
Click More Comments： 0 times.
Click More Replies： 1 times.
Click More Replies： 2 times.
This post can not expand
Time Log: 2019-05-29 14:22:42

------------------
Dealing with: https://www.facebook.com/tstartel/videos/589662731510587/
This post can not expand
Time Log: 2019-05-29 14:22:47

------------------
Dealing with: https://www.facebook.com/tstartel/posts/3151580078201351
Click More Comments： 0 times.
Click Mor

Click More Replies： 3 times.
Click More Replies： 4 times.
Click More Replies： 5 times.
Click See More ： 1 times.
Click See More ： 2 times.
Expand Succed!
Update PostInfo complete!
Update CommentsInfo complete!
Time Log: 2019-05-29 14:29:38

------------------
Dealing with: https://www.facebook.com/tstartel/photos/a.413576055335114/2997171090308918/
Click More Comments： 0 times.
Click More Replies： 1 times.
Click More Replies： 2 times.
Click More Replies： 3 times.
Expand Succed!
Update PostInfo complete!
Update CommentsInfo complete!
Time Log: 2019-05-29 14:29:53

------------------
Dealing with: https://www.facebook.com/tstartel/posts/3000021670023860
Click More Comments： 0 times.
Click More Replies： 1 times.
Click More Replies： 2 times.
Click More Replies： 3 times.
Click More Replies： 4 times.
Click More Replies： 5 times.
Click More Replies： 6 times.
Click More Replies： 7 times.
Click More Replies： 8 times.
Click More Replies： 9 times.
Click More Replies： 10 times.
Click See More ： 1 

C:\Users\TL_Yu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


This post can not expand
Time Log: 2019-05-29 14:32:44

------------------
Dealing with: https://www.facebook.com/chtmobile/photos/a.264830403546609/2722488054447486/
Expand Succed!
Update PostInfo complete!
Update CommentsInfo complete!
Time Log: 2019-05-29 14:32:55

------------------
Dealing with: https://www.facebook.com/chtmobile/photos/a.264830403546609/2717455341617424/
Expand Succed!
Update PostInfo complete!
Update CommentsInfo complete!
Time Log: 2019-05-29 14:33:07

------------------
Dealing with: https://www.facebook.com/chtmobile/photos/a.264830403546609/2715269541836004/
Expand Succed!
Update PostInfo complete!
Update CommentsInfo complete!
Time Log: 2019-05-29 14:33:18

------------------
Dealing with: https://www.facebook.com/chtmobile/photos/a.264830403546609/2707938675902424/
Click More Replies： 1 times.
Expand Succed!
Update PostInfo complete!
Update CommentsInfo complete!
Time Log: 2019-05-29 14:33:30

------------------
Dealing with: https://www.facebook.com/chtmo

Click See More ： 1 times.
Expand Succed!
Update PostInfo complete!
Update CommentsInfo complete!
Time Log: 2019-05-29 14:39:20

------------------
Dealing with: https://www.facebook.com/chtmobile/photos/a.264830403546609/2133054030057561/
Click More Comments： 0 times.
Click More Replies： 1 times.
Click More Replies： 2 times.
Click More Replies： 3 times.
Click More Replies： 4 times.
Click More Replies： 5 times.
Click More Replies： 6 times.
Click More Replies： 7 times.
Click More Replies： 8 times.
Click More Replies： 9 times.
Click More Replies： 10 times.
Click More Replies： 11 times.
Click More Replies： 12 times.
Click See More ： 1 times.
Expand Succed!
Update PostInfo complete!
Update CommentsInfo complete!
Time Log: 2019-05-29 14:39:40

------------------
Dealing with: https://www.facebook.com/notes/%E4%B8%AD%E8%8F%AF%E9%9B%BB%E4%BF%A1%E8%A1%8C%E5%8B%95%E5%AD%B8%E5%9C%92/hami-%E8%80%81%E5%B8%AB%E7%9A%84%E6%9C%89%E7%8D%8E%E5%BE%B5%E7%AD%94%E4%B8%AD%E7%8D%8E%E5%90%8D%E5%96%AE/2714190525

...

Click More Comments： 0 times.
Expand Succed!
Update PostInfo complete!
Update CommentsInfo complete!
Time Log: 2019-05-29 19:59:11

------------------
Dealing with: https://www.facebook.com/Aptg.tw/photos/a.165348830171434/2362074067165555/
Click More Comments： 0 times.
Expand Succed!
Update PostInfo complete!
Update CommentsInfo complete!
Time Log: 2019-05-29 19:59:24

------------------
Dealing with: https://www.facebook.com/Aptg.tw/photos/a.165348830171434/2361927933846835/
Expand Succed!
Update PostInfo complete!
Update CommentsInfo complete!
Time Log: 2019-05-29 19:59:36

------------------
Dealing with: https://www.facebook.com/Aptg.tw/photos/a.165348830171434/2361926790513616/
This post can not expand
Time Log: 2019-05-29 19:59:40

------------------
Dealing with: https://www.facebook.com/Aptg.tw/photos/a.165348830171434/2359861214053507/
Expand Succed!
Update PostInfo complete!
Update CommentsInfo complete!
Time Log: 2019-05-29 19:59:51

------------------
Dealing with: https:/

Expand Succed!
Update PostInfo complete!
Update CommentsInfo complete!
Time Log: 2019-05-29 20:06:47

------------------
Dealing with: https://www.facebook.com/Aptg.tw/posts/2342673525772276
Click More Comments： 0 times.
Click More Comments： 1 times.
Click More Comments： 2 times.
Click More Comments： 3 times.
Click More Comments： 4 times.
Click More Comments： 5 times.
Click More Comments： 6 times.
Click More Comments： 7 times.
Click More Comments： 8 times.
Click More Comments： 9 times.
Click More Comments： 10 times.
Click More Comments： 11 times.
Click More Replies： 1 times.
Click More Replies： 2 times.
Click See More ： 1 times.
Expand Succed!
Update PostInfo complete!
Update CommentsInfo complete!
Time Log: 2019-05-29 20:07:33

------------------
Dealing with: https://www.facebook.com/Aptg.tw/photos/a.165348830171434/2342235329149429/
Click More Comments： 0 times.
Click More Comments： 1 times.
Click More Comments： 2 times.
Click More Comments： 3 times.
Click More Comments： 4 times.
Cl

Click More Comments： 7 times.
Click More Comments： 8 times.
Click More Comments： 9 times.
Click More Comments： 10 times.
Click More Comments： 11 times.
Click More Comments： 12 times.
Click More Comments： 13 times.
Click More Comments： 14 times.
Click More Comments： 15 times.
Click More Comments： 16 times.
Click More Comments： 17 times.
Click More Replies： 1 times.
Expand Succed!
Update PostInfo complete!
Update CommentsInfo complete!
Time Log: 2019-05-29 20:15:22

------------------
Dealing with: https://www.facebook.com/Aptg.tw/photos/a.698345903538388/2321070824599213/
Click More Comments： 0 times.
Click More Comments： 1 times.
Click More Comments： 2 times.
Click More Comments： 3 times.
Click More Comments： 4 times.
Click More Comments： 5 times.
Click More Comments： 6 times.
Click More Comments： 7 times.
Click More Comments： 8 times.
Click More Comments： 9 times.
Click More Comments： 10 times.
Click More Comments： 11 times.
Click More Comments： 12 times.
Click More Comments： 13 times

Click More Comments： 38 times.
Click More Comments： 39 times.
Click More Comments： 40 times.
Click More Comments： 41 times.
Click More Comments： 42 times.
Click More Comments： 43 times.
Click More Comments： 44 times.
Click More Comments： 45 times.
Click More Comments： 46 times.
Click More Comments： 47 times.
Click More Comments： 48 times.
Click More Comments： 49 times.
Click More Comments： 50 times.
Click More Replies： 1 times.
Click More Replies： 2 times.
Click More Replies： 3 times.
Click See More ： 1 times.
Expand Succed!
Update PostInfo complete!
Update CommentsInfo complete!
Time Log: 2019-05-29 20:20:52

------------------
Dealing with: https://www.facebook.com/Aptg.tw/photos/a.165348830171434/2292870207419275/
Click More Replies： 1 times.
Expand Succed!
Update PostInfo complete!
Update CommentsInfo complete!
Time Log: 2019-05-29 20:21:04

------------------
Dealing with: https://www.facebook.com/Aptg.tw/photos/a.165348830171434/2292858177420478/
Click More Comments： 0 times.
Cli

Click More Comments： 3 times.
Click More Comments： 4 times.
Click More Comments： 5 times.
Click More Comments： 6 times.
Click More Comments： 7 times.
Click More Comments： 8 times.
Expand Succed!
Update PostInfo complete!
Update CommentsInfo complete!
Time Log: 2019-05-29 20:28:37

------------------
Dealing with: https://www.facebook.com/Aptg.tw/photos/a.165348830171434/2286335764739386/
This post can not expand
Time Log: 2019-05-29 20:28:40

------------------
Dealing with: https://www.facebook.com/Aptg.tw/photos/a.165348830171434/2287777637928532/
Click More Comments： 0 times.
Click More Comments： 1 times.
Click More Comments： 2 times.
Click More Comments： 3 times.
Click More Comments： 4 times.
Click More Comments： 5 times.
Click More Replies： 1 times.
Click See More ： 1 times.
Click See More ： 2 times.
Expand Succed!
Update PostInfo complete!
Update CommentsInfo complete!
Time Log: 2019-05-29 20:29:08

------------------
Dealing with: https://www.facebook.com/Aptg.tw/posts/228801817

In [16]:
Posts = Posts.reset_index(drop=True)
Posts = Posts.loc[:,['Name', 'ID', 'Time', 'Content', 'Comments', 'Shares', 'Likes', 'Loves', 'Hahas', 'Wows', 'Sads', 'Angrys', 'Updatetime', 'Link']]
Posts

,Name,ID,Time,Content,Comments,Shares,Likes,Loves,Hahas,Wows,Sads,Angrys,Updatetime,Link
0,台灣之星,tstartel,2019-05-24 03:33,過去20年，行動通訊服務不停演進 從2G、3G到4G，到現在人人都在談的5G 你的生活形...,394,14,316,7,8,0,0,0,2019-05-29 20:34,https://www.facebook.com/tstartel/videos/11002...
1,台灣之星,tstartel,2019-05-26 21:00,本月最強🔥全民一起辦$299/$499🔥 新申辦/攜碼 #4G上網+#網內通話 #雙飽 最後...,432,8,148,5,0,2,0,0,2019-05-29 20:33,https://www.facebook.com/tstartel/videos/23007...
2,台灣之星,tstartel,2019-05-27 21:18,停停停⛔STOP⛔國文老師要森77了喔😤 管它「省」字怎麼念 #門號搭手機 #00C編 就要...,60,5,368,4,14,0,0,0,2019-05-29 20:32,https://www.facebook.com/tstartel/photos/a.854...
3,台灣之星,tstartel,2019-05-27 02:00,這個價格你會買嗎?那為什麼粉絲會❓ #U姐編 秒殺PM提品🥊 LINE好友沒有專屬優惠我不同...,65,2,275,5,0,3,0,0,2019-05-29 20:32,https://www.facebook.com/tstartel/photos/a.413...
4,亞太電信Gt智慧生活,Aptg.tw,2019-01-25 01:00,#追蹤亞太電信粉絲團長知識 各個國家的錢幣大有不同，但你有聽過0元紙鈔嗎? 印度曾經為了防止...,2,2,23,0,2,3,0,0,2019-05-29 20:31,https://www.facebook.com/Aptg.tw/photos/a.1653...
5,亞太電信Gt智慧生活,Aptg.tw,2019-01-24 01:00,【春節出國趣🛫國際漫遊吃到飽現在96元起】 寒假就是要開心出國~亞太電信陪你迎豬年🎊 4G月...,3,0,31,0,0,2,0,0,2019-05-29 20:31,https://www.facebook.com/Aptg.tw/photos/a.1653...
6,亞太電信Gt智慧生活,Aptg.tw,2019-01-24 23:00,網美打卡第六站來到👉#烘爐地 一探中和景色的最佳視野就在這裡了，還可以順便祈求運勢平安好方便...,5,0,17,0,0,0,0,1,2019-05-29 20:31,https://www.facebook.com/Aptg.tw/posts/2273778...
7,亞太電信Gt智慧生活,Aptg.tw,2019-01-24 19:30,【#亞太電信魔速方塊 公司用戶來實證】 Gt特派員 Regina' 薛蓓蓓又來囉~ 一起來看...,5,13,23,1,0,0,0,2,2019-05-29 20:31,https://www.facebook.com/Aptg.tw/videos/212104...
8,台灣之星,tstartel,2019-05-28 04:24,蝦密！世界有名的 #嚴島神社 #大鳥居 竟然要整修一年😱醬去日本有什麼意義😭 唉唷～日本這麼...,21,2,321,8,0,1,0,0,2019-05-29 20:31,https://www.facebook.com/tstartel/photos/a.854...
9,亞太電信Gt智慧生活,Aptg.tw,2019-01-28 01:00,#追蹤亞太電信粉絲團長知識 台灣著名打卡景點超級多，但粉絲們知道在北車內有隱藏一個日本人瘋傳...,4,0,22,0,0,0,0,1,2019-05-29 20:30,https://www.facebook.com/Aptg.tw/photos/a.1653...


In [17]:
Comments = Comments.reset_index(drop=True)
Comments = Comments.loc[:,['ID', 'Name', 'Time', 'Content', 'RepID', 'RepName', 'Link', 'Updatetime']]
Comments

,ID,Name,Time,Content,RepID,RepName,Link,Updatetime
0,360044337354953,台灣之星,2019-05-29 20:32,請確認私訊唷~,100003553136627,林悅,https://www.facebook.com/tstartel/videos/11002...,2019-05-29 20:34
1,360044337354953,台灣之星,2019-05-24 22:52,小編不許你胡說~~~,100000436789882,葉大昌,https://www.facebook.com/tstartel/videos/11002...,2019-05-29 20:34
2,360044337354953,台灣之星,2019-05-24 22:05,來~~請跟小編私訊這樣做🤗,100000129379951,Ching Kit Ho,https://www.facebook.com/tstartel/videos/11002...,2019-05-29 20:34
3,100009597651095,Hank Chen,2019-05-24 21:57,老客戶要如何299終身吃到飽,1100239263492734:9:0,台灣之星,https://www.facebook.com/tstartel/videos/11002...,2019-05-29 20:34
4,360044337354953,台灣之星,2019-05-24 22:20,老客戶來看私訊~~小編說了什麼吧🤩,100009597651095,Hank Chen,https://www.facebook.com/tstartel/videos/11002...,2019-05-29 20:34
5,100034741457030,吳世凱,2019-05-24 22:04,小編真的是多才多藝，身兼多職，要請這種當小編最適合,1100239263492734:9:0,台灣之星,https://www.facebook.com/tstartel/videos/11002...,2019-05-29 20:34
6,360044337354953,台灣之星,2019-05-24 22:23,謝謝世凱支持肯定💛我們會一直努力的😘,100034741457030,吳世凱,https://www.facebook.com/tstartel/videos/11002...,2019-05-29 20:34
7,100019171691847,Lilylily Wu,2019-05-24 22:15,老客戶要299終身吃到飽+1,1100239263492734:9:0,台灣之星,https://www.facebook.com/tstartel/videos/11002...,2019-05-29 20:34
8,360044337354953,台灣之星,2019-05-24 22:37,續約朋友來看看私訊吧🤓,100019171691847,Lilylily Wu,https://www.facebook.com/tstartel/videos/11002...,2019-05-29 20:34
9,100034741457030,吳世凱,2019-05-24 23:02,Lilylily Wu ➕1,100019171691847,Lilylily Wu,https://www.facebook.com/tstartel/videos/11002...,2019-05-29 20:34


# 保存資料

In [18]:
Posts.to_pickle('C:/Users/TL_Yu/Desktop/Posts.plk')
Comments.to_pickle('C:/Users/TL_Yu/Desktop/Comments.plk')

In [19]:
# Posts.to_csv('C:/Users/TL_Yu/Desktop/Posts.csv')
# Comments.to_csv('C:/Users/TL_Yu/Desktop/Comments.csv')

# To Do List
- 故障(無法expand的貼文要避免加入貼文清單或者加入後，要讓下次再來爬文，不能直接跳過。)
 - 評估將加載更多留言放在同一個function中

# 維護測試區

In [60]:
driver = webdriver.Chrome()
driver.get('https://www.facebook.com/')
time.sleep(1)
driver.find_element_by_partial_link_text('English').click()

In [61]:
driver.get('https://www.facebook.com/tstartel/videos/1100239263492734/?permPage=1')
#time.sleep(0.7)
#PostExpand()

In [62]:
ClosePopup()

In [63]:
#ClickOldest()

In [64]:
#driver.execute_script('window.scrollTo(20, document.body.scrollHeight);')
MoreComments()

Click More Comments： 0 times.
Click More Comments： 1 times.
Click More Comments： 2 times.


In [21]:
ClosePopup()
ClickOldest()
MoreComments()
MoreReplies()
SeeMore()

Click More Comments： 0 times.
Click More Comments： 1 times.
Click More Comments： 2 times.
Click More Comments： 3 times.
Click More Comments： 4 times.
Click More Comments： 5 times.
Click More Comments： 6 times.
Click More Comments： 7 times.
Click More Comments： 8 times.
Click More Comments： 9 times.
Click More Comments： 10 times.
Click More Comments： 11 times.
Click More Comments： 12 times.
Click More Comments： 13 times.
Click More Comments： 14 times.
Click More Comments： 15 times.
Click More Comments： 16 times.
Click More Comments： 17 times.
Click More Comments： 18 times.
Click More Comments： 19 times.
Click More Comments： 20 times.
Click More Comments： 21 times.
Click More Comments： 22 times.
Click More Comments： 23 times.
Click More Comments： 24 times.
Click More Comments： 25 times.
Click More Comments： 26 times.
Click More Comments： 27 times.
Click More Comments： 28 times.
Click More Comments： 29 times.
Click More Comments： 30 times.


In [19]:
MoreComments()

Click More Comments： 0 times.
Click More Comments： 1 times.


In [ ]:
MoreReplies()

In [ ]:
SeeMore()

In [7]:
# 關閉彈窗
def ClosePopup():
    time.sleep(1)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(1)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    ClickNotNow()
    driver.find_element_by_xpath('//div[@class="_5pcr userContentWrapper"]//a[@data-testid="UFI2CommentsCount/root"]').click()
    time.sleep(1)
    ClickNotNow()
    time.sleep(0.5)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(0.5)
    ClickNotNow()
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(0.5)
    ClickNotNow()

# 選擇按照New或Oldest來顯示留言(選擇最相關或所有留言都不會會真的顯示所有留言)
def ClickOldest():
    time.sleep(1)
    driver.find_element_by_xpath('//a[@data-testid="UFI2ViewOptionsSelector/link"]').click()
    time.sleep(1)
    try:
        driver.find_element_by_partial_link_text('Comments shown in chronological order with the oldest comments at the top.').click()
    except:
        try:
            driver.find_element_by_partial_link_text('New comments and those with new replies appear at the top.').click()
        except:
            print('Plz, Check this post arragne type!')
    time.sleep(1)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    ClickNotNow()
    time.sleep(1)

# 偵測是否有「更多留言」（第一層），若有則點擊
def MoreComments():
    k = len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]'))
    # 卡住的次數超過100次就停止
    l = 0
    while (k != 0) & (l <= 300):
        print('Click More Comments： ' + str(l) + ' times.')
        try:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            driver.find_element_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]').click()
            time.sleep(1)
            # 若彈出註冊視窗點擊not now!
            ClickNotNow()
            k = len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]'))
        except:
            time.sleep(0.1)
        finally:
            l += 1
    time.sleep(1)

# 偵測是否有「更多回覆」（第二層），若有則點擊
def MoreReplies():
    k = len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_1"]'))
    # 卡住的次數超過100次就停止
    l = 0
    while (k != 0) & (l <= 300):
        l += 1
        print('Click More Replies： ' + str(l) + ' times.')
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        driver.find_element_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_1"]').click()
        time.sleep(0.5)
        k = len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_1"]'))
    time.sleep(1)
    
# 偵測是否有「展開留言」（第二層留言內容），若有則點擊
def SeeMore():
    k = len(driver.find_elements_by_xpath('//a[@class="_5v47 fss"]'))
    # 卡住的次數超過100次就停止
    l = 0
    while (k != 0) & (l <= 1000):
        l += 1
        print('Click See More ： ' + str(l) + ' times.')
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        driver.find_element_by_xpath('//a[@class="_5v47 fss"]').click()
        time.sleep(0.5)
        k = len(driver.find_elements_by_xpath('//a[@class="_5v47 fss"]'))
    time.sleep(1)
    
# 組合以上函數，將貼文內容展開
def PostExpand():
    ClosePopup()
    # ClickOldest() # 展開留言會有無法加載更多留言的問題，暫時先不使用
    MoreComments()
    MoreReplies()
    SeeMore()

In [26]:
soup = BeautifulSoup(driver.page_source)
CommentsInfo(soup)

,ID,Name,Time,Content,RepID,RepName,Link,Updatetime
0,100008036701674,陳國進,2019-05-26 12:44,我今天要去屏東市😆😆😆,360044337354953,台灣之星,https://www.facebook.com/tstartel/photos/a.854...,2019-05-27 01:14
1,360044337354953,台灣之星,2019-05-26 13:10,要做好防曬唷!!!,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.854...,2019-05-27 01:14
2,100000293657248,賴清溪,2019-05-26 13:16,貼心😏,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.854...,2019-05-27 01:14
3,360044337354953,台灣之星,2019-05-26 13:28,🤗,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.854...,2019-05-27 01:14
4,100008036701674,陳國進,2019-05-26 17:26,賴清溪 真的很貼心呢～,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.854...,2019-05-27 01:14
5,100008036701674,陳國進,2019-05-26 17:26,台灣之星 好哦～謝謝小編,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.854...,2019-05-27 01:14
6,360044337354953,台灣之星,2019-05-26 17:35,如何~今天有沒有看見什麼有趣的事呢~~~國進~~~,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.854...,2019-05-27 01:14
7,100008036701674,陳國進,2019-05-26 19:33,台灣之星 沒有~~但以上速度超穩~~,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.854...,2019-05-27 01:14
8,360044337354953,台灣之星,2019-05-26 19:41,期待你分享小祕密唷(〃ω〃),100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.854...,2019-05-27 01:14
9,100008036701674,陳國進,2019-05-26 19:48,台灣之星 下載80.5 上傳45.0,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.854...,2019-05-27 01:14


In [24]:
soup = BeautifulSoup(driver.page_source)

Crawl Comments Failed!


,ID,Name,Time,Content,RepID,RepName,Link
0,100008036701674,陳國進,2019-05-26 12:44,我今天要去屏東市😆😆😆,360044337354953,台灣之星,https://www.facebook.com/tstartel/photos/a.854...


In [56]:
PostInfo(soup)

,Name,ID,Time,Content,Comments,Shares,Likes,Loves,Hahas,Wows,Sads,Angrys,Updatetime,Link
0,台灣之星,tstartel,2019-05-15 22:11,台灣之星2019全新品牌主張 你的電信就該是這個樣子⚡今日登場 全民一起辦活動⚡同步開跑‼️...,1700,88,732,12,0,0,0,20,2019-05-27 00:43,https://www.facebook.com/tstartel/posts/327440...


In [57]:
CommentsInfo(soup)

,ID,Name,Time,Content,RepID,RepName,Link,Updatetime
0,100004563705267,李維尼,2019-05-16 13:14,Line 好友不是說299只到今天上午09:59。,360044337354953,台灣之星,https://www.facebook.com/tstartel/posts/327440...,2019-05-27 00:44
1,360044337354953,台灣之星,2019-05-16 13:25,我們聽到大家的心聲~加碼到5/31唷!!!!,100004563705267,李維尼,https://www.facebook.com/tstartel/posts/327440...,2019-05-27 00:44
2,100003595572374,魏坤輝,2019-05-16 13:33,網頁進不去,100004563705267,李維尼,https://www.facebook.com/tstartel/posts/327440...,2019-05-27 00:44
3,360044337354953,台灣之星,2019-05-16 13:57,小編測試可以唷~~http://bit.ly/2HihMLD,100004563705267,李維尼,https://www.facebook.com/tstartel/posts/327440...,2019-05-27 00:44
4,100004563705267,李維尼,2019-05-16 14:03,台灣之星 請問我去年有辦188了，但188是21m吃到飽，現在299是不限速吃到飽，我可以去...,100004563705267,李維尼,https://www.facebook.com/tstartel/posts/327440...,2019-05-27 00:44
5,360044337354953,台灣之星,2019-05-16 14:23,老客戶免費勁速體驗底家~快去申請體驗吧~https://doc.tstartel.com/e...,100004563705267,李維尼,https://www.facebook.com/tstartel/posts/327440...,2019-05-27 00:44
6,100004563705267,李維尼,2019-05-16 14:26,台灣之星 謝謝，已申請，OK的話，馬上來辦。,100004563705267,李維尼,https://www.facebook.com/tstartel/posts/327440...,2019-05-27 00:44
7,100004563705267,李維尼,2019-05-16 14:39,台灣之星 看來我家是各大電信悲劇，競速是23.9/11.6，和188吃到飽，差不多。,100004563705267,李維尼,https://www.facebook.com/tstartel/posts/327440...,2019-05-27 00:44
8,360044337354953,台灣之星,2019-05-16 14:50,行動網路容易受使用地點附近的房屋密集度、建築裝潢使用材質、附近使用人數多寡…等等因素影響，我...,100004563705267,李維尼,https://www.facebook.com/tstartel/posts/327440...,2019-05-27 00:44
9,100001860123628,Wu Hung Chuan,2019-05-16 13:15,499沒有比較划算，你們的費率本來就超過1分1元，會打到499的算起來還是499為什麼要辦4...,360044337354953,台灣之星,https://www.facebook.com/tstartel/posts/327440...,2019-05-27 00:44


In [55]:
Posts = UpdateData(DateFrame_o = Posts, DateFrame_n = nPost)
Posts

,Name,ID,Time,Content,CommentCounts,Shares,Like,Love,Haha,Wow,Sad,Angry,Updatetime,Link
2,台灣之星,360044337354953,2019-05-18 21:00,好想去日本🇯🇵看繡球花唷 小編們聽 #U姐編 嘟嘟囔囔一週快煩死了 交換個眼色，作戰計劃開始...,55,13,626,8,2,0,0,0,2019-05-20 23:47,https://www.facebook.com/tstartel/photos/a.413...
0,台灣之星,360044337354953,2019-05-19 02:00,#週1福利日 #每週一1230 #週週登場 有沒有人跟 #00C編 一樣 手機沒電會狂症發作...,13,4,299,3,0,5,0,0,2019-05-20 23:46,https://www.facebook.com/tstartel/photos/a.413...
1,台灣之星,360044337354953,2019-05-19 20:52,反正只要一兩塊錢 買個袋子裝一裝，比較方便啦😆 環保餐具用完還要洗，好麻煩 直接拿竹筷和塑膠...,35,13,79,1,0,0,3,0,2019-05-20 23:45,https://www.facebook.com/tstartel/posts/328473...


In [56]:
Comments = UpdateData(DateFrame_o = Comments,DateFrame_n = nComments)
Comments

,ID,Name,Time,Content,RepID,RepName,Link,Updatetime
48,360044337354953,台灣之星,2019-05-19 15:52,看美美照片一樣心情會hen好,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
61,360044337354953,台灣之星,2019-05-19 19:40,img,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
70,100008036701674,陳國進,2019-05-20 17:18,台灣之星 還好我16:00回到家過一下才下雨~~,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
69,360044337354953,台灣之星,2019-05-20 16:48,有沒有記得帶雨傘呀~,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
68,100008036701674,陳國進,2019-05-20 16:44,台灣之星 小編，下大雨了,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
67,360044337354953,台灣之星,2019-05-19 21:08,加油加油(๑˃̵ᴗ˂̵)ﻭ其實上課是很幸福D~~晚安😉😉,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
66,100008036701674,陳國進,2019-05-19 21:03,台灣之星 明天又要上課，小編晚安~~,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
65,360044337354953,台灣之星,2019-05-19 20:47,早點洗洗睡明天又是新的一天~(✿◠‿◠),100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
64,100008036701674,陳國進,2019-05-19 20:42,台灣之星 是啊～,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
63,360044337354953,台灣之星,2019-05-19 20:41,國進到家啦❓,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47


In [16]:
driver.quit()

# 障礙清單
https://www.facebook.com/taiwanmobile/photos/a.1515817481972021/2484327321787694/
https://www.facebook.com/Aptg.tw/videos/639447693147224/
https://www.facebook.com/Aptg.tw/videos/422586768321086/



# 其他

In [18]:
# 原本想要登入的方式獲取連結，但FB會偵測異常行為，暫時先不登入
# 關閉Chrome的「通知」提醒
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-notifications")

driver = webdriver.Chrome(options=chrome_options)
url = 'https://www.facebook.com/'
driver.get(url)
time.sleep(3)

# 啟動瀏覽器並登入Facebook
username = driver.find_element_by_id('email')
username.send_keys('61034b001@gms.ndhu.edu.tw')
passwd=driver.find_element_by_id('pass')
passwd.send_keys('19920309')
button=driver.find_element_by_id('loginbutton')
button.click()